In [2]:
import numpy as np
from elasticsearch import helpers, Elasticsearch
import csv
import pandas as pd
import os
import time
from tqdm import tqdm
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from inflection import singularize
import matplotlib.pyplot as plt

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.info()


{'name': 'm-c02wv1jnhtd5',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'fpFxjw6cRjyGQOMl6ThLdw',
 'version': {'number': '7.5.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'e9ccaed468e2fac2275a3761849cbee64b39519f',
  'build_date': '2019-11-26T01:06:52.518245Z',
  'build_snapshot': False,
  'lucene_version': '8.3.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [ ]:
es.indices.delete(index = 'std_ref')

In [ ]:
def text_preprocessing(text):
    text= text.lower()
    text= re.sub(r'[^a-z]',' ',text)
    text= " ".join([s for s in text.split() if len(s)>2])
    text= " ".join([x for x in text.split() if x not in stopwords.words('english')])
    text= " ".join([inflection.singularize(x) for x in text.split()])
    text= ' '.join(text.split())
    return(text)

In [ ]:
file_path = '.../souradip/file_text.csv'
df_text = pd.read_csv('.../souradip/file_text.csv')


In [ ]:
df_text['abstracts'] = df_food['abstracts'].apply(lambda x: text_preprocessing(x))


#### The process of Indexing

In [ ]:
index='index_abstract'

In [ ]:
def doc_generator(data,index):
    df_iter= data.iterrows()
    
    for index,document in df_iter:
        yield{
            "_index": 'index_abstract',
            "_type": "_doc",
            "_id" : f"{index}",
            "_source": document.to_json()
        }
    raise StopIteration
    

In [ ]:
for success,info in tqdm(helpers.parallel_bulk(es,doc_generator(df_text,index))):
    if not success:
        print('A document failed:', info)
es.indices.refresh(index=index)

In [ ]:
def SEARCH(text,index,field):
    
    res=es.search(index=index,body={"query":{"match":{field:{"query":text,"operator":"or","fuzziness": "0"
                                                            }}}},size = 15)
    
    
    return([(x.get('_source'),x.get('_score')) for x in res['hits']['hits']])



In [ ]:
def query_generator(df):
    while True:
        yield (df_query['question'])

In [ ]:
query_generator(df_query)

In [ ]:
import concurrent.futures

In [ ]:
start = time.time()
queries=[]
with concurrent.futures.ThreadPoolExecutor() as executor:
    queries={executor.submit(SEARCH, query,index,'abstracts'): query for query in df_query['question']}
print(time.time()-start)


In [ ]:
#queries
#SEARCH('corn','std_ref','qarth_product_name')

In [ ]:
out = [x.result() for x in queries]